In [1]:
import pandas as pd
import sqlite3
import os

In [2]:
qtd_msg = 1

lista = list(range(qtd_msg))
print(lista)

[0]


In [3]:
qtd_msg = 1

lista = range(qtd_msg)
print(lista)

range(0, 1)


In [4]:
qtd_msg = 4
lista = list(range(qtd_msg - 1, -1, -1))
for p, i in enumerate(lista):
    print("posição", i)
    print("indice", p+1)
    

posição 3
indice 1
posição 2
indice 2
posição 1
indice 3
posição 0
indice 4


In [5]:
base_clientes = pd.read_csv('../data/base_clientes.csv')

In [6]:
base_clientes.head()

,cpf,nome,segmento,qtd_cont,vlr_total_div,max_dias_atraso
0,42006925890,Bryan Campos,Van Gogh,10,5000,60
1,34366343878,Juan Moura,Private,5,2000,30
2,72388905850,Ana Sophia da Cruz,Van Gogh,1,2293,28
3,85044167398,Marina Moraes,Especial,7,2685,42
4,86406862031,Marcos Vinicius Cardoso,Van Gogh,13,9173,50


In [7]:
# Especificar o caminho para o diretório onde o banco de dados está localizado
path = "../data/"

# Especificar o valor do CPF como uma string entre aspas simples
cpf_value = 42006925890
query = f"SELECT * FROM customer_database_table WHERE cpf = '{cpf_value}'"

# Usar a declaração `with` para gerenciar a conexão ao banco de dados SQLite
with sqlite3.connect(os.path.join(path, 'customer_database.sqlite')) as conn:
    # Executar a consulta SQL e ler o resultado em um DataFrame do pandas
    base = pd.read_sql_query(query, conn)

In [8]:
base.head()

,cpf,nome,segmento,qtd_cont,vlr_total_div,max_dias_atraso
0,42006925890,Bryan Campos,Van Gogh,10,5000,60


In [9]:
base["nome"].values

array(['Bryan Campos'], dtype=object)

In [14]:
from pymongo import MongoClient

In [17]:
client = MongoClient('mongodb://localhost:27017/', serverSelectionTimeoutMS=5000)

In [20]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, serverselectiontimeoutms=5000)

In [18]:
def conectar_mongo():
    try:
        client = MongoClient('mongodb://localhost:27017/', serverSelectionTimeoutMS=5000)
        client.server_info()  # Gera uma exceção se não conseguir conectar
        print("Conexão com MongoDB bem-sucedida.")
        if client:
            db = client['meu_banco_de_dados']
            collection = db['clientes_conversas']
        return client, db, collection
    except Exception as e:
        print(f"Erro ao conectar ao MongoDB: {e}")
        return None

In [19]:
client, db, collection = conectar_mongo() # Conectar ao banco de dados

Erro ao conectar ao MongoDB: localhost:27017: [WinError 10061] Nenhuma conexão pôde ser feita porque a máquina de destino as recusou ativamente (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 5.0s, Topology Description: <TopologyDescription id: 667c35440f2a7d5a65d143e1, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [WinError 10061] Nenhuma conexão pôde ser feita porque a máquina de destino as recusou ativamente (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>


TypeError: cannot unpack non-iterable NoneType object

In [ ]:
import json
from bson import ObjectId

In [52]:
# Função personalizada para serializar ObjectId
class JSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, ObjectId):
            return str(obj)
        return json.JSONEncoder.default(self, obj)

In [136]:
def formatar_json(documento):
    return json.dumps(documento, indent=4, ensure_ascii=False, cls=JSONEncoder)

In [137]:
# Consultar todos os documentos na coleção
def consultar_todos():
    documentos = collection.find()
    for doc in documentos:
        print(formatar_json(doc))

# Consultar documentos com uma condição
def consultar_condicao(cpf):
    documento = collection.find_one({"cpf": cpf})
    if documento:
        formatar_json(documento)
    else:
        print(f"Nenhum documento encontrado para o CPF {cpf}")

In [173]:
def inserir_cliente(cpf, dados_cliente, dados_oferta, assunto, dt_hr_ini):
    cliente = collection.find_one({"cpf": cpf})
    # Inserir novo cliente se não existir
    if not cliente:
        collection.insert_one({
            "cpf": cpf,
            # "dados_cliente": dados_cliente,
            # "dados_oferta": dados_oferta,
            "conversas": [
                {
                    "assunto": assunto,
                    "chats": [
                        {
                            "data_hora_inicio": dt_hr_ini,
                            # "mensagens": []
                        }
                    ]
                }
            ]
        })
    else:
        # Inserir nova conversa caso o cliente já existir na base
        assunto_existe = collection.find_one({"cpf": cpf, "conversas.assunto": assunto})
        if not assunto_existe:
            print("assunto novo")
            # Adicionar um novo assunto com o chat inicial
            collection.update_one(
                {"cpf": cpf},
                {"$push": {
                    "conversas": {
                        "assunto": assunto,
                        "chats": [
                            {
                                "data_hora_inicio": dt_hr_ini,
                                "mensagens": []
                            }
                        ]
                    }
                }}
            )
        else:
            # Adicionar um novo chat ao assunto existente
            collection.update_one(
                {"cpf": cpf, "conversas.assunto": assunto},
                {"$push": {
                    "conversas.$.chats": {
                        "data_hora_inicio": dt_hr_ini,
                        "mensagens": []
                    }
                }}
            )


In [178]:
cpf = 34343434343
assunto = "Reclamação"
dt_hr_ini = "2024-06-27"

In [179]:
dados_cliente = {
    "nome": "João da Silva",
    "email": "caique@gmail.com",
    "telefone": "11999999999"
}

In [180]:
dados_oferta = {
    "data_ref": "2024-06-27",
    "produto": "Smartphone",
    "preco": 1500.00,
    "desconto": 100.00
}

In [185]:
inserir_cliente(cpf, dados_cliente, dados_oferta, assunto, dt_hr_ini)

In [186]:
consultar_todos()
# consultar_condicao(42006925890)

{
    "_id": "66771c88b59999d3c1c2f55f",
    "cpf": 34343434343,
    "dados_cliente": {
        "nome": "João da Silva",
        "email": "caique@gmail.com",
        "telefone": "11999999999"
    },
    "dados_oferta": {
        "data_ref": "2024-06-27",
        "produto": "Smartphone",
        "preco": 1500.0,
        "desconto": 100.0
    },
    "conversas": [
        {
            "assunto": "Reclamação",
            "chats": [
                {
                    "data_hora_inicio": "2024-06-27"
                }
            ]
        }
    ]
}


In [183]:
collection.delete_many({})

DeleteResult({'n': 1, 'ok': 1.0}, acknowledged=True)

In [191]:
def inserir_mensagem(cpf, assunto, dt_hr_ini, dict_mensagem):
    client, db, collection = conectar_mongo()  # Conectar ao banco de dados
    collection.update_one(
        {"cpf": cpf},
        {"$push": {"conversas.$[conversa].chats.$[chat].mensagens": dict_mensagem}},
        array_filters=[
            {"conversa.assunto": assunto},
            {"chat.data_hora_inicio": dt_hr_ini}
        ]
    )

In [195]:
dict_mensagem = {
    "id": 1,
    "data_hora": "2024-06-27 10:30:00",
    "mensagem_cliente": "Iphone 24",
    "sugestao_ia": "Esse produto não existe",
    "rating_sugestao_ia": 4,
    "resposta_final_operador": "Sr. João, o produto correto é o Iphone 15",
}

In [196]:
inserir_mensagem(cpf, assunto, dt_hr_ini, dict_mensagem)

Conexão com MongoDB bem-sucedida.


In [197]:
consultar_todos()

{
    "_id": "66771c88b59999d3c1c2f55f",
    "cpf": 34343434343,
    "dados_cliente": {
        "nome": "João da Silva",
        "email": "caique@gmail.com",
        "telefone": "11999999999"
    },
    "dados_oferta": {
        "data_ref": "2024-06-27",
        "produto": "Smartphone",
        "preco": 1500.0,
        "desconto": 100.0
    },
    "conversas": [
        {
            "assunto": "Reclamação",
            "chats": [
                {
                    "data_hora_inicio": "2024-06-27",
                    "mensagens": [
                        {
                            "id": 1,
                            "data_hora": "2024-06-27 10:00:00",
                            "mensagem_cliente": "Olá, gostaria de saber mais sobre o produto.",
                            "sugestao_ia": "Qual produto, Sr. João?",
                            "rating_sugestao_ia": 5,
                            "resposta_final_operador": "Sobre qual produto o Sr gostaria de saber mais?"
       

### Verificando se deu certo o armazenamento no código original